In [1]:
# -
#
# SPDX-FileCopyrightText: Copyright (c) 2024 Pietro Carlo Boldini, Rene Pecnik and the CUBENS contributors. All rights reserved.
# SPDX-License-Identifier: MIT
#
# -

import math as m
import cmath as cm
import numpy as np
import scipy.optimize as opt
from scipy.interpolate import interp1d
from numpy import linalg as npla

from scipy.sparse.linalg import spsolve
from scipy.sparse        import diags, hstack, vstack
from scipy.integrate     import solve_bvp

from functools import partial
import CoolProp.CoolProp as CP # CoolProp (http://www.coolprop.org/) for first approximation

import matplotlib.pyplot as plt
from matplotlib import rc, rcParams

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text', usetex=True)
rcParams.update({'font.size': 18})
rcParams['figure.figsize']   = [8,6]
rcParams['mathtext.fontset'] = 'stix'
rcParams['font.family']      = 'STIXGeneral'

# Main class for CHA calculation

In [2]:
class BL():

    def writeToFile(self):
        file = open("inputDNS/initCHA_params.h", "w")
        file.write("!------------ use equation of state ------------------ \n")
        file.write('USE_EOS  = "VdW" \n')
        file.write("!------------ set non-dimensional reference values for computation ------------------ \n")
        file.write("Pra   = %.10f \n" % self.Pr)
        file.write("Ec   = %.10e \n" % self.Ec)
        file.write("Ma   = %.10e \n" % self.Ma)
        file.write("eos_dof   = %.6f \n" % self.dof)
        file.write("eos_ac   = %.6f \n" % self.omega_ac)
        file.write("Ri_wall   = %.6f \n" % self.Ri_wall)
        file.write("Ri_unit   = %.6f \n" % self.Ri_unit)
        file.write("!------------ set dimensional values for computation ------------------ \n")
        file.write("Tcrit   = %.6f \n" % self.T_crit)
        file.write("Pcrit   = %.6f \n" % self.P_crit)
        file.write("Vcrit   = %.6e \n" % self.V_crit)
        file.write("eos_Rgas   = %.6f \n" % self.Rg)
        file.write("! ------------ set wall BC for computation ------------------ \n")
        file.write("Twall_bot   = %.6f \n" % self.Twall_bot)
        file.write("Twall_top   = %.6f \n" % self.Twall_top)
        file.write("!------------ set reference values for computation ------------------ \n")
        file.write("Tref   = %.10f \n" % self.Tred)
        file.write("Pref   = %.10f \n" % self.Pred)
        file.write("Rhoref   = %.10f \n" % self.Rhored)
        file.write("Cpref   = %.10f \n" % self.CpR)
        file.write("SOSref   = %.10f \n" % self.sos) 
        file.write("Rref   = %.10f \n" % self.R) 
        file.write("! ----------- set viscosity and conductivity ---------------- \n")
        file.write('USE_VISC  = "%s" \n' % self.visc_bc)
        if self.visc_bc == 'JST':
            file.write("Muref   = %.10e \n" % self.mu_inf)
            file.write("Kref   = %.10e \n" % self.ka_inf)
        file.close()
        
    def showParameters(self):
        
        print("\nDNS parameters:\n")
        print("USE_EOS = VdW ")
        print("Pra = ", self.Pr)
        print("Ec = ", self.Ec)
        print("Ma = ", self.Ma)
        print("Ri_wall = ", self.Ri_wall)
        print("Ri_unit = ", self.Ri_unit)
        print("Pref = ", self.Pred)
        print("Cv/R= = ", self.CvR)
        print('USE_VISC  = ', self.visc_bc)
        
        print('\nDNS initial conditions are saved in ./inputDNS/')
        print('DNS parameters saved in ./inputDNS/')

# Different EoS

## Van der Waals

In [3]:
class CHA_VdW(BL): 
    def __init__(self, Ec = None, Twall_bot=None, Twall_top=None, Pred =None, Tred =None, Pr = None, CvR=None, visc = None, Ri_wall = None):   
        self.Ec = Ec   
        self.Twall_bot = Twall_bot
        self.Twall_top = Twall_top
        self.Pred = Pred
        self.Tred = Tred 
        self.CvR = CvR
        self.f_vdw()
        rho0 = CP.PropsSI("D", "P", Pred*self.P_crit, "T", Tred*self.T_crit, self.fluid)/self.Rho_crit # this is just an approximation
        self.Rhored = float(opt.fsolve(self.f_rh,rho0,args=(Tred)))
        self.Hred = self.f_h(self.Rhored, self.Tred)
        self.CpR = self.f_cp(self.Rhored, self.Tred, self.CvR) 
        self.sos = self.f_sos(self.Rhored, self.Tred, self.CvR)
        self.Ma=(self.Ec*self.CpR*self.Tred/self.sos**2/self.Zc)**0.5
        self.Pr = Pr
        self.dof = 9  
        self.Ri_wall = Ri_wall
        self.Ri_unit = Ri_wall/(1/self.Twall_bot-1)
        if visc == 'Constant':
            self.visc_bc = 'Constant'
            self.mu_inf = self.Tred
            self.ka_inf = self.mu_inf
        elif visc == 'JST':
            self.visc_bc = 'JST'
            [self.mu_inf,self.ka_inf] = self.f_muka(self.Rhored, self.Tred, visc)
    
    def f_vdw(self):
        self.Zc=3/8
        self.R=1/self.Zc
        self.a=3
        self.b=1/3
        self.Ru=8.31451
        self.fluid="CO2"
        self.omega_ac=0.224
        self.dof=9
        self.M=CP.PropsSI("molemass",self.fluid)
        self.Rg=self.Ru/self.M         
        self.T_crit=CP.PropsSI("Tcrit",self.fluid) 
        self.P_crit=CP.PropsSI("Pcrit",self.fluid)
        self.V_crit=self.Zc*self.Rg*self.T_crit/self.P_crit
        self.Rho_crit=1/self.V_crit
        return self
            
    def f_rh_T(self, x, h):
        T, rho = x
        F1 = (8 * T) / (3 * (1 / rho) - 1) - 3 * rho**2 - self.Pred
        F2 = self.R * self.CvR * T - self.a * rho + self.Pred / rho - h * self.Hred
        return np.array([F1, F2])
    
    def f_rh(self, rho, T):
        F1 = (8 * T) / (3 * (1 / rho) - 1) - 3 * rho ** 2 - self.Pred
        return np.array(F1)
    
    def f_h(self, rho, T):
        return self.R*self.CvR*T - self.a*rho + self.Pred/rho
    
    def f_cp(self, rho, T, CvR):
        v=1/rho
        return CvR+1/(1-(3*v-1)**2/(4*T*v**3))
    
    def f_sos(self, rho, T, CvR):
        v=1/rho
        return ( (1+1/CvR)*self.R*T*(3*v/(3*v-1))**2-6/v )**0.5

    def f_muka(self, rho, T, visc):
        if visc == 'Constant':
            return np.maximum(T,1.0e-6), np.maximum(T,1.0e-6)
        elif visc == 'JST':
            # viscosity
            mu_1 = np.where(T <= 1.50,
                            34 * 10**(-5) * T**0.94,
                            17.78 * 10**(-5) * (4.58 * T - 1.67)**(5/8))
            f_rho = 0.10230 + 0.023364 * rho + 0.058533 * rho**2 - 0.040758 * rho**3 + 0.0093324 * rho**4
            mu_diff = (f_rho**4 - 10**(-4))   
            kappa_factor=(0.307*self.CvR+0.539)
            kappa_EU = 15 / 4 * self.Ru * mu_1 * kappa_factor
            kappa_RT_tri = 6.54 * 10**(-5) * (np.exp(0.2826 * T) - 1 / np.exp(0.3976 * T**2))
            kappa = np.zeros_like(rho)
            mask1 = rho < 0.50
            mask2 = (rho >= 0.50) & (rho < 2.0)
            mask3 = rho >= 2.0
            # conductivity
            f_rho1 = 14.0 * (np.exp(0.535 * rho) - 1)
            kappa_diff1 = (f_rho1 * 10**(-8)) / self.Zc**5
            kappa[mask1] = kappa_diff1[mask1] * (4.1868 / 10**(-2)) + kappa_EU[mask1]
            f_rho2 = 13.1 * (np.exp(0.67 * rho) - 1.069)
            kappa_diff2 = (f_rho2 * 10**(-8)) / self.Zc**5
            kappa[mask2] = kappa_diff2[mask2] * (4.1868 / 10**(-2)) + kappa_EU[mask2]
            f_rho3 = 2.976 * (np.exp(1.155 * rho) + 2.016)
            kappa_diff3 = (f_rho3 * 10**(-8)) / self.Zc**5
            kappa[mask3] = kappa_diff3[mask3] * (4.1868 / 10**(-2)) + kappa_EU[mask3]
            return mu_diff + mu_1, kappa
    

# Solve channel with Van der Waals fluid

In [4]:
# Input parameters of CHA_VdW:
# 1) Reference Eckert number (Mach number will be calculated): Ec = u^2_b/(Cp_ref T_ref)
# 2) Ratio: bottom wall temperature to reference temperature
# 3) Ratio: top wall temperature to reference temperature
# 4) Reference reduced pressure (p_r = p_ref/p_crit)
# 5) Reference reduced temperature  (T_r = T_ref/T_crit)
# 6) Reference Prandtl number 
# 7) Cv/R - ratio
# 8) Viscosity law: 'JST' (JossiStielThodos) or 'Constant'

vdw = CHA_VdW(Ec=0.004,Twall_bot=1.0,Twall_top=1.0,Pred=1.1,Tred=0.9,Pr=1.0,CvR=9/2,visc='Constant',Ri_wall=0)
vdw.writeToFile()
vdw.showParameters()


DNS parameters:

USE_EOS = VdW 
Pra =  1.0
Ec =  0.004
Ma =  0.10034653141716765
Pref =  1.1
Cv/R= =  4.5
USE_VISC  =  JST

DNS initial conditions are saved in ../inputDNS/
DNS parameters saved in ../inputDNS/
